In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/twitch-users-dataset/Twitch_user_data.csv',index_col='Channel')

In [ ]:
data.info()

Channel: Name of the channel

Watch Time: No. of minutes watched in minutes

Stream Time: No.of minutes live streamed in minutes

Peak viewers: Total number of viewers at the peak time

Average Viewers: No. of viewers watched on average

Followers: Totak number of followers on their channe
l
Followers gained: No.of followers gained in this time

Views Gained: No. of views gained

Partenered: was the video partnered or collaborated with someone?

Mature: Was the video had mature content?

Language: Language in which video was streamed

In [ ]:
data.head()

In [ ]:
data.describe()

### Change column names and remove some space

In [ ]:
for i in data.columns:
    if ' ' in i:
        data = data.rename(columns={i:i.replace(' ','_')})
for i in data.columns:
    if '(' in i:
        data = data.rename(columns={i:i.replace('(','_')})
for i in data.columns:
    if ')' in i:
        data = data.rename(columns={i:i.replace(')','_')}) 

In [ ]:
data.head()

Let's see for some null values[](http://)

In [ ]:
null_cols = [i for i in data.columns if data[i].isnull().any()]
null_cols

No nan value found

## Label encoding the bool columns

In [ ]:
data['Partnered']=data.Partnered.map(lambda p:int(p))
data['Mature']=data.Mature.map(lambda p:int(p))
data.head()

## Language Analysis

Let's find total no. of languages that our data has

In [ ]:
data.Language.nunique()

In [ ]:
plot = data.groupby('Language').Average_viewers.agg(['max','min','mean','count']).sort_values(by='max',ascending=False)
plot

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x=plot.index,y=plot['max'])
plt.title('Max Average views Vs Language')
plt.show()

In [ ]:
plot2 = data.groupby('Language').Peak_viewers.agg(['max','min','mean','count']).sort_values(by='max',ascending=False)
plot2

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x=plot2.index,y=plot2['max'])
plt.title('Peak viewers Vs Language')
plt.show()

English speaking world, Okay!! 

In [ ]:
plt.figure(figsize=(14,6))
sns.scatterplot(y=data['Language'], x=data['Average_viewers'])

seems like some outliers there lets find out about their owners, they might be big channels

In [ ]:
data.groupby('Channel').Average_viewers.min().sort_values(ascending=False).head(10)

* So they are dota2ti and dota2ti_ru lets see their straming time

In [ ]:
data.groupby('Channel').Stream_time_minutes_.max().sort_values().head(10)

Okay so dota2ti and dota2ti_ru have like least stream time but alot of average viewers, the might just have fake views!!

In [ ]:
plt.figure(figsize=(14,6))
sns.scatterplot(x=data['Stream_time_minutes_'], y=data['Average_viewers'])

that clearly are outliers, so many average viewers with less streaming time

Seems like we need a lot of data cleaning for out liers

In [ ]:
# Defining function for removing outliers
def rem_outliers(data,col,outlier):
    q1,q2=np.percentile(col,[25,75])
    iqr = q2-q1
    lb = q1 - 1.5*iqr
    ub = q2 + 1.5*iqr
    for i in range(len(col)):
        if(col[i]>ub or col[i]<lb):
            if data.index[i] not in outlier:
                outlier.append(data.index[i])

In [ ]:
outlier=[]
for col in data.columns:
    if col != 'Language' and col!='Partnered' and col!='Mature':
        rem_outliers(data,data[col],outlier)

In [ ]:
len(outlier)
outlier

In [ ]:
data_cleaned=data.drop(outlier,axis=0,errors='ignore')

In [ ]:
data_cleaned

In [ ]:
plt.figure(figsize=(14,6))
sns.scatterplot(x=data_cleaned['Stream_time_minutes_'], y=data_cleaned['Average_viewers'])

now it looks fine ;-)

In [ ]:
plot3 = data_cleaned.groupby('Language').Average_viewers.agg(['max','min','mean','count']).sort_values(by='max',ascending=False)
plot3

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x=plot3.index,y=plot3['max'])
plt.title('Average_viewers Vs Language')
plt.show()

WELL WELL WELL how the turn tables, Koreans overtook english as max average viewers

In [ ]:
plot4 = data_cleaned.groupby('Language').Peak_viewers.agg(['max','min','mean','count']).sort_values(by='max',ascending=False)
plot4

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x=plot4.index,y=plot4['max'])
plt.title('Peak viewers Vs Language')
plt.show()

English still ahead in peak time viewers!!

In [ ]:
'shroud' in data_cleaned.index

## Let's see who's topping followers list

In [ ]:
data_cleaned.groupby('Channel').Followers.max().sort_values(ascending=False).head(10)

## Alalysis based on Partnered and matured content

In [ ]:
data_cleaned[data_cleaned['Partnered']==1].mean()

In [ ]:
data_cleaned[data_cleaned['Partnered']==0].mean()

It's clearly evident that Partnering and collaborating helped gain followers and watch time more, but on average viewers prefered watching non-partenered more.

Partenered channels have more followers on avg



In [ ]:
data_cleaned[data_cleaned['Mature']==1].mean()

In [ ]:
data_cleaned[data_cleaned['Mature']==0].mean()

## Comparison B/W English and Russian Langs


In [ ]:
data_cleaned[(data_cleaned['Language']=='English')].mean()

In [ ]:
data_cleaned[(data_cleaned['Language']=='Russian')].mean()

English has a definite advantage of being universal language gaining more followers and watch time on avg but russian channels are in the rise getting more no. followers than english